# `uarray` NumPy Compatability

In [2]:
from uarray import *
import numpy as np
from numba import njit

TypeError: issubclass() arg 1 must be a class

## Original Expression

Let's look at this simple NumPy expression of calling the outer production of two values and then indexing it:

In [ ]:
def some_fn(a, b):
    return np.multiply.outer(a, b)[5]

We can see that this does a lot of extra work, since we discard most of the results of the outer product after indexing. We can look at the time:

In [ ]:
args = [np.arange(1000), np.arange(10)]

In [ ]:
%time some_fn(*args)

## Uarray reduced

Now let's use uarray's `optimize` decorator to create an updated function that specifes the dimensionality of the arrays to produced an optimized form:

In [ ]:
@optimize
def optimized_some_fn(a, b):
    return some_fn(a.has_dim(1), b.has_dim(1))

Now let's try our function out to see if it's faster:

In [ ]:
%time optimized_some_fn(*args)

Yep about 10x as fast. Let's look at how this is done! First, we create an abstract representation of the array operations:

In [ ]:
optimized_some_fn.__optimize_steps__['resulting_expr']

Then, we compile that to Python AST:

In [ ]:
print(optimized_some_fn.__optimize_steps__['ast_as_source'])

## Numba optimized

To give this an extra speed boost, we can compile the returned expression with Numba:

In [ ]:
numba_optimized = njit(optimized_some_fn)

In [ ]:
# run once first to compile
numba_optimized(*args)
                
%time numba_optimized(*args)

Great, another 2x speedup!

In [ ]:
# ast.dump(ast.parse("(1,) + ()"))

## Unkown dimensionality?

What if we want to produce a version of the function that works on any dimensional input? Or if we just want to actually defer to NumPy's implementation and not replace `outer`? We simply omit the `with_dim` methods and we get back an abstract representation that is compiled without any knowledge of the dimensionality:

In [ ]:
some_fn

In [ ]:
dims_not_known = optimize(some_fn)

In [ ]:
dims_not_known.__optimize_steps__['resulting_expr']

In [ ]:
print(dims_not_known.__optimize_steps__['ast_as_source'])